In [ ]:
import numpy as np
import pandas as pd
import math
from tqdm import tqdm
from BidderModel.Bid import bid, robust_bid_ce
from Utils.NoiseCTR import noise_CTR_ce
from Utils.Vizualization import plot_results_with_std,visualize_comparison


In [ ]:
def safe_mean(arr):
    return np.mean(arr) if len(arr) > 0 else 0

def safe_std(arr):
    return np.std(arr) if len(arr) > 0 else 0
    
def experiment_1(SEED,epsilon,robustness,version):
    I = 10
    T = 100 
    np.random.seed(SEED)
    
    CTRmin=0.01
    CTRmax=0.1
    CVR = np.random.uniform(0.1, 0.9, (I, T))
    CTR_real,CTR_noised= noise_CTR_ce(I,T,epsilon,SEED,CTRmin,CTRmax,10000)
    E = np.ones((I, T))
    
    c=np.log(CTR_noised)-np.log(E-CTR_noised)
    d=-np.log(E-CTR_noised)

    B = np.full(I, 1.0)
    C = np.full(I, 1.0)
    
    alpha=0

    is_win = np.zeros((I, T))
    BID = np.zeros((I, T))
    p = np.ones((I, T))
    q = np.ones((I, T))
    gamma = np.ones((I, T))
    beta = np.ones((I, T))
    theta = np.ones((I, T))
    delta = np.ones((I, T))
    wp = np.zeros(T)
    current_budgets= np.tile(B.reshape(-1, 1), T)
    
    N_resolve=1
    N_solve=10

    bids = np.zeros(I)
    for t in tqdm(range(T)):
        if t <1:
            bids =np.array([min(np.random.uniform(0, c_i), b_i) 
                            for c_i, b_i in zip(C, current_budgets[:,t])])         
        else:
            pqmax=1.0e-7
            for i in range(I):  
                if robustness=='non-robust': 
                    bids[i],p[i,t],q[i,t] = bid(CTR_noised[i,:t], CVR[i,:t], wp[:t], B[i], alpha, C[i], 
                                  is_win[i,:t], np.sum(is_win[i,:t]),pqmax,p[i,t-1],q[i,t-1],N_resolve,N_solve)
                    bids[i]=min(bids[i],current_budgets[i,t])
                else:   
                    bids[i],gamma[i,t],beta[i,t],theta[i,t],delta[i,t] =robust_bid_ce(CTR_noised[i,:t+1], CVR[i,:t+1], np.append(wp[:t], wp[t-1]),current_budgets[i,max(0,t-N_solve)], epsilon, C[i], 
                                      np.append(is_win[i,:t], is_win[i,t-1]), t+1,c[i,:t+1],d[i,:t+1],gamma[i,t-1],beta[i,t-1],theta[i,t-1],delta[i,t-1],N_resolve,N_solve)
                    bids[i]=min(bids[i],current_budgets[i,t])
                if bids[i]==-1:
                    bids[i]=min(BID[i,t-1],current_budgets[i,t])
                    gamma[i,t],beta[i,t],theta[i,t],delta[i,t]=gamma[i,t-1],beta[i,t-1],theta[i,t-1],delta[i,t-1]
    
        BID[:,t] = bids
        winner = np.argmax(bids)
        wp[t] = bids[winner]
        is_win[winner,t] = 1

        current_budgets[winner,t+1:] -= wp[t]

        reserve_price=0.01
        if bids[winner]>reserve_price:
            wp[t] = bids[winner]
            is_win[winner,t] = 1
            current_budgets[winner,t+1:] -= wp[t]
        else:
            wp[t] =reserve_price

    total_conversions = np.sum(CTR_real * CVR * is_win)
    cheap_clicks_ratio = []
    for i in range(I):
        
        if np.sum(is_win[i]) > 0:
            cheap_clicks = np.sum((BID[i]/CTR_real[i] < C[i]) * CTR_real[i] * is_win[i])
            total_clicks = np.sum(CTR_real[i] * is_win[i])
            cheap_clicks_ratio.append(cheap_clicks/total_clicks if total_clicks > 0 else 0)
    avg_cheap_clicks = safe_mean(cheap_clicks_ratio)
    
    total_cost = np.sum(BID * is_win)
    total_clicks = np.sum(CTR_real * is_win)
    avg_cpc = total_cost / total_clicks if total_clicks > 0 else 0
    return (total_conversions,avg_cheap_clicks*100,avg_cpc)

In [ ]:
def run_experiments(robust_type):
    results = []
    all_results=[]
  
    for epsilon in [20,20.5,21,21.5,22,22.5,23,23.5,24]:

        print(epsilon)
        
        conversions = []
        clicks = []
        cpcs = []
       
        
        for SEED in range(0,10):
            total_conversions, avg_cheap_clicks, avg_cpc = experiment_1(SEED, epsilon, robust_type, 'synthetic')
            conversions.append(total_conversions)
            clicks.append(avg_cheap_clicks)
            cpcs.append(avg_cpc)
            
            all_results.append({
                'epsilon': epsilon,
                'conversions': total_conversions,
                'cheap_clicks': avg_cheap_clicks,
                'cpc': avg_cpc,
                'SEED':SEED
            })
     
        results.append({
            'epsilon': epsilon,
            'total_conversions_mean': safe_mean(conversions),
            'total_conversions_std': safe_std(conversions),
            'avg_cheap_clicks_mean': safe_mean(clicks),
            'avg_cheap_clicks_std': safe_std(clicks),
            'avg_cpc_mean': safe_mean(cpcs),
            'avg_cpc_std': safe_std(cpcs)
        })

    df = pd.DataFrame(results)
    df.to_csv(f'Results_synthetic_{robust_type}_ce.csv', index=False)

    df1 = pd.DataFrame(all_results)
    df1.to_csv(f'All_results_synthetic_{robust_type}_ce.csv', index=False)

run_experiments('non-robust')
run_experiments('robust')

In [ ]:
plot_results_with_std('Results_synthetic_non-robust_ce.csv','Results_synthetic_robust_ce.csv','Results_synthetic_ce.png')